In [ ]:
import csv
import pandas as pd
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import math
import numpy as np

In [ ]:
!pip install Sastrawi

     |████████████████████████████████| 215kB 6.1MB/s 


In [ ]:
df = pd.read_excel('../../traindata.xlsx')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_text  233 non-null    object
 1   Class       233 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.8+ KB


In [ ]:
df

,tweet_text,Class
0,@hendi_clean @jokowi Lah.. gini aja sekalinya ...,-1
1,2 MENTERI TERSANGKA KPK BOSSS MALUU @jokowi,-1
2,@jokowi Kenapa setiap saya baca atau dengar pa...,-1
3,"@kurawa @jokowi Kur, dia mungkin lagi kejar ta...",-1
4,@Lukmanbunker @AhmadAi36504178 @jokowi Masa ha...,-1
...,...,...
228,@TofaTofa_id @jokowi yg mengganas habis tangka...,-1
229,BANGSAT! @KemenBUMN @KemenkesRI @jokowi @eric...,-1
230,@Yatie84991237 @kulispiselo @tanca_ra @ganefri...,-1
231,Yg lain sulit hidup tanpa teknologi hanya si ...,-1


In [ ]:
df1 = pd.read_excel('../../testdata.xlsx')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_text  104 non-null    object
 1   Class       104 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


In [ ]:
df1

,tweet_text,Class
0,@faizalassegaf @jokowi Hehehe..... Menurutku ...,1
1,@fadjroeL @jokowi @aniesbaswedan Cuitanmu sang...,-1
2,@Dennysiregar7 @jokowi Kalau diam..bilang lema...,-1
3,@PSI_Jakarta @jokowi Halahhhh... Goblok ngeles...,-1
4,@Ard1nawan @josuwa11147411 @rha_tjoensimatu @j...,-1
...,...,...
99,@jokowi Semoga sehat selalu Pak @jokowi dan se...,1
100,@jokowi makasih bapak Jokowi yang sudah meluan...,1
101,@N0N4m3_90 Dasar manusia pembawa sial!!!!!! @...,-1
102,Semoga selalu menjadi yang terbaik dan terdepa...,1


In [ ]:
class data:
  
  def __init__ (self, text, val = 0) :
    self.text = text
    self.val = val


In [ ]:
stopwords = ['a','ada','adalah','adanya','adapun','agak','agaknya','agar','akan','akankah','akhir',
            'akhiri','akhirnya','aku','akulah','amat','amatlah','anda','andalah','antar','antara',
            'antaranya','apa','apaan','apabila','apakah','apalagi','apatah','arti','artinya','asal',
            'asalkan','atas','atau','ataukah','ataupun','awal','awalnya','b','bagai','bagaikan',
            'bagaimana','bagaimanakah','bagaimanapun','bagainamakah','bagi','bagian','bahkan','bahwa',
            'bahwasannya','bahwasanya','baik','baiklah','bakal','bakalan','balik','banyak','bapak',
            'baru','bawah','beberapa','begini','beginian','beginikah','beginilah','begitu','begitukah',
            'begitulah','begitupun','bekerja','belakang','belakangan','belum','belumlah','benar',
            'benarkah','benarlah','berada','berakhir','berakhirlah','berakhirnya','berapa','berapakah',
            'berapalah','berapapun','berarti','berawal','berbagai','berdatangan','beri','berikan',
            'berikut','berikutnya','berjumlah','berkali-kali','berkata','berkehendak','berkeinginan',
            'berkenaan','berlainan','berlalu','berlangsung','berlebihan','bermacam','bermacam-macam',
            'bermaksud','bermula','bersama','bersama-sama','bersiap','bersiap-siap','bertanya',
            'bertanya-tanya','berturut','berturut-turut','bertutur','berujar','berupa','besar',
            'betul','betulkah','biasa','biasanya','bila','bilakah','bisa','bisakah','boleh','bolehkah',
            'bolehlah','buat','bukan','bukankah','bukanlah','bukannya','bulan','bung','c','cara',
            'caranya','cukup','cukupkah','cukuplah','cuma','d','dahulu','dalam','dan','dapat','dari',
            'daripada','datang','dekat','demi','demikian','demikianlah','dengan','depan','di','dia',
            'diakhiri','diakhirinya','dialah','diantara','diantaranya','diberi','diberikan','diberikannya',
            'dibuat','dibuatnya','didapat','didatangkan','digunakan','diibaratkan','diibaratkannya',
            'diingat','diingatkan','diinginkan','dijawab','dijelaskan','dijelaskannya','dikarenakan',
            'dikatakan','dikatakannya','dikerjakan','diketahui','diketahuinya','dikira','dilakukan',
            'dilalui','dilihat','dimaksud','dimaksudkan','dimaksudkannya','dimaksudnya','diminta',
            'dimintai','dimisalkan','dimulai','dimulailah','dimulainya','dimungkinkan','dini','dipastikan',
            'diperbuat','diperbuatnya','dipergunakan','diperkirakan','diperlihatkan','diperlukan',
            'diperlukannya','dipersoalkan','dipertanyakan','dipunyai','diri','dirinya','disampaikan',
            'disebut','disebutkan','disebutkannya','disini','disinilah','ditambahkan','ditandaskan',
            'ditanya','ditanyai','ditanyakan','ditegaskan','ditujukan','ditunjuk','ditunjuki','ditunjukkan',
            'ditunjukkannya','ditunjuknya','dituturkan','dituturkannya','diucapkan','diucapkannya',
            'diungkapkan','dong','dua','dulu','e','empat','enak','enggak','enggaknya','entah','entahlah',
            'f','g','guna','gunakan','h','hadap','hai','hal','halo','hallo','hampir','hanya','hanyalah',
            'hari','harus','haruslah','harusnya','helo','hello','hendak','hendaklah','hendaknya','hingga',
            'i','ia','ialah','ibarat','ibaratkan','ibaratnya','ibu','ikut','ingat','ingat-ingat','ingin',
            'inginkah','inginkan','ini','inikah','inilah','itu','itukah','itulah','j','jadi','jadilah',
            'jadinya','jangan','jangankan','janganlah','jauh','jawab','jawaban','jawabnya','jelas',
            'jelaskan','jelaslah','jelasnya','jika','jikalau','juga','jumlah','jumlahnya','justru',
            'k','kadar','kala','kalau','kalaulah','kalaupun','kali','kalian','kami','kamilah','kamu',
            'kamulah','kan','kapan','kapankah','kapanpun','karena','karenanya','kasus','kata','katakan',
            'katakanlah','katanya','ke','keadaan','kebetulan','kecil','kedua','keduanya','keinginan',
            'kelamaan','kelihatan','kelihatannya','kelima','keluar','kembali','kemudian','kemungkinan',
            'kemungkinannya','kena','kenapa','kepada','kepadanya','kerja','kesampaian','keseluruhan',
            'keseluruhannya','keterlaluan','ketika','khusus','khususnya','kini','kinilah','kira',
            'kira-kira','kiranya','kita','kitalah','kok','kurang','l','lagi','lagian','lah','lain',
            'lainnya','laku','lalu','lama','lamanya','langsung','lanjut','lanjutnya','lebih','lewat',
            'lihat','lima','luar','m','macam','maka','makanya','makin','maksud','malah','malahan',
            'mampu','mampukah','mana','manakala','manalagi','masa','masalah','masalahnya','masih',
            'masihkah','masing','masing-masing','masuk','mata','mau','maupun','melainkan','melakukan',
            'melalui','melihat','melihatnya','memang','memastikan','memberi','memberikan','membuat',
            'memerlukan','memihak','meminta','memintakan','memisalkan','memperbuat','mempergunakan',
            'memperkirakan','memperlihatkan','mempersiapkan','mempersoalkan','mempertanyakan','mempunyai',
            'memulai','memungkinkan','menaiki','menambahkan','menandaskan','menanti','menanti-nanti',
            'menantikan','menanya','menanyai','menanyakan','mendapat','mendapatkan','mendatang','mendatangi',
            'mendatangkan','menegaskan','mengakhiri','mengapa','mengatakan','mengatakannya','mengenai',
            'mengerjakan','mengetahui','menggunakan','menghendaki','mengibaratkan','mengibaratkannya',
            'mengingat','mengingatkan','menginginkan','mengira','mengucapkan','mengucapkannya','mengungkapkan',
            'menjadi','menjawab','menjelaskan','menuju','menunjuk','menunjuki','menunjukkan','menunjuknya',
            'menurut','menuturkan','menyampaikan','menyangkut','menyatakan','menyebutkan','menyeluruh',
            'menyiapkan','merasa','mereka','merekalah','merupakan','meski','meskipun','meyakini','meyakinkan',
            'minta','mirip','misal','misalkan','misalnya','mohon','mula','mulai','mulailah','mulanya','mungkin',
            'mungkinkah','n','nah','naik','namun','nanti','nantinya','nya','nyaris','nyata','nyatanya',
            'o','oleh','olehnya','orang','p','pada','padahal','padanya','pak','paling','panjang','pantas',
            'para','pasti','pastilah','penting','pentingnya','per','percuma','perlu','perlukah','perlunya',
            'pernah','persoalan','pertama','pertama-tama','pertanyaan','pertanyakan','pihak','pihaknya',
            'pukul','pula','pun','punya','q','r','rasa','rasanya','rupa','rupanya','s','saat','saatnya','saja',
            'sajalah','salam','saling','sama','sama-sama','sambil','sampai','sampai-sampai','sampaikan','sana',
            'sangat','sangatlah','sangkut','satu','saya','sayalah','se','sebab','sebabnya','sebagai',
            'sebagaimana','sebagainya','sebagian','sebaik','sebaik-baiknya','sebaiknya','sebaliknya',
            'sebanyak','sebegini','sebegitu','sebelum','sebelumnya','sebenarnya','seberapa','sebesar',
            'sebetulnya','sebisanya','sebuah','sebut','sebutlah','sebutnya','secara','secukupnya','sedang',
            'sedangkan','sedemikian','sedikit','sedikitnya','seenaknya','segala','segalanya','segera',
            'seharusnya','sehingga','seingat','sejak','sejauh','sejenak','sejumlah','sekadar','sekadarnya',
            'sekali','sekali-kali','sekalian','sekaligus','sekalipun','sekarang','sekaranglah','sekecil',
            'seketika','sekiranya','sekitar','sekitarnya','sekurang-kurangnya','sekurangnya','sela','selain',
            'selaku','selalu','selama','selama-lamanya','selamanya','selanjutnya','seluruh','seluruhnya',
            'semacam','semakin','semampu','semampunya','semasa','semasih','semata','semata-mata','semaunya',
            'sementara','semisal','semisalnya','sempat','semua','semuanya','semula','sendiri','sendirian',
            'sendirinya','seolah','seolah-olah','seorang','sepanjang','sepantasnya','sepantasnyalah',
            'seperlunya','seperti','sepertinya','sepihak','sering','seringnya','serta','serupa','sesaat',
            'sesama','sesampai','sesegera','sesekali','seseorang','sesuatu','sesuatunya','sesudah',
            'sesudahnya','setelah','setempat','setengah','seterusnya','setiap','setiba','setibanya',
            'setidak-tidaknya','setidaknya','setinggi','seusai','sewaktu','siap','siapa','siapakah',
            'siapapun','sini','sinilah','soal','soalnya','suatu','sudah','sudahkah','sudahlah','supaya',
            't','tadi','tadinya','tahu','tak','tambah','tambahnya','tampak','tampaknya','tandas','tandasnya',
            'tanpa','tanya','tanyakan','tanyanya','tapi','tegas','tegasnya','telah','tempat','tentang','tentu',
            'tentulah','tentunya','tepat','terakhir','terasa','terbanyak','terdahulu','terdapat','terdiri',
            'terhadap','terhadapnya','teringat','teringat-ingat','terjadi','terjadilah','terjadinya','terkira',
            'terlalu','terlebih','terlihat','termasuk','ternyata','tersampaikan','tersebut','tersebutlah',
            'tertentu','tertuju','terus','terutama','tetap','tetapi','tiap','tiba','tiba-tiba','tidak',
            'tidakkah','tidaklah','tiga','toh','tuju','tunjuk','turut','tutur','tuturnya','u','ucap','ucapnya',
            'ujar','ujarnya','umumnya','ungkap','ungkapnya','untuk','usah','usai','v','w','waduh','wah','wahai',
            'waktunya','walau','walaupun','wong','x','y','ya','yaitu','yakin','yakni','yang', 'yg','z', 'amp', 'jokowi']

In [ ]:
train =[]

for row in range(len(df)):
  train.append(data(df.tweet_text[row], df.Class[row]))

In [ ]:
test =[]

for row in range(len(df1)):
  test.append(data(df1.tweet_text[row]))

In [ ]:
test[0].val 

0

In [ ]:
def cleaning(text):
  cleaned = []
  for i in range(len(text)):
    text[i] = re.sub (r"http\S+", " ", text[i])
    temp = re.sub (r"([^a-zA-Z]+)", " " , text[i])
    cleaned.append(temp)
  return cleaned


In [ ]:
def casefolding(text):
  foldedcase = []
  for i in text:
    temp = i.lower()
    foldedcase.append(temp)
  return foldedcase

In [ ]:
def token(text):
  token = []
  for i in text:
    temp = i.split()
    token.append(temp)
  return token

In [ ]:
def filtering(text, stopwords):
  filtered = []
  for i in range(len(text)):
    temp = []
    for token in text[i]:
      if token not in stopwords:
        temp.append(token)
    filtered.append(temp)
  return filtered

In [ ]:
def stemming(text, stemmer):

  for i in range(len(text)):
    for token in range(len(text[i])):
      text[i][token] =stemmer.stem(text[i][token])

  return text

In [ ]:
def preprocessing(data):
  text = []
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  for i in range(len(data)):
    text.append(data[i].text)
  clean = cleaning(text)
  casefold = casefolding(clean)
  tokenize = token(casefold)
  filtered = filtering(tokenize, stopwords)
  stemm = stemming(filtered, stemmer)
  return stemm

In [ ]:
def preprocesslex(data):
  text = []
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  for i in range(len(data)):
    text.append(data[i].text)
  clean = cleaning(text)
  casefold = casefolding(clean)
  tokenize = token(casefold)
  return tokenize

In [ ]:
hasiltest = preprocessing(test)

In [ ]:
hasiltest

In [ ]:
hasiltrain = preprocessing(train)

In [ ]:
hasiltrain

In [ ]:
lex = train+test
doc_tweet = preprocesslex(lex)

In [ ]:
def fitur_list(text):
  fitur =[]

  for token in text:
    for w in token:
      if w not in fitur:
        fitur.append(w)
  return fitur

In [ ]:
def termfreq(fitur, text):
  termfreq = []
  
  for term in fitur:
    temp = []
    for doc in text:
      temp.append(doc.count(term))
    termfreq.append(temp)
  return termfreq
      

In [ ]:
def weightTF(termfreq):
  weight_tf = []
  for i in range(len(termfreq)):
    temp = []
    for n in range(len(termfreq[i])):
      wtf = 0
      if termfreq[i][n]>0:
        wtf = 1+ math.log10(termfreq[i][n])
      else:
        wtf = 0
      temp.append(wtf)
    weight_tf.append(temp)
  return weight_tf



In [ ]:
def idf(weightedTF):
  df = []
  idf = []
  for i in range(len(weightedTF)):
    temp = 0
    for j in range(len(weightedTF[i])): 
      if weightedTF[i][j] > 0:
        temp +=1
    df.append(temp)
  for i in range(len(df)):
    temp = math.log10((len(weightedTF[i])/df[i]))
    idf.append(temp)
  return idf

In [ ]:
def tf_idf(idf, weightedTF):
  tfidf = []
  for i in range(len(idf)):
    tf_idf = []
    for j in range(len(weightedTF[i])):
      if weightedTF[i][j]>0:
        temp = idf[i] * weightedTF[i][j]
        tf_idf.append(temp)
      else:
        tf_idf.append(0)
    tfidf.append(tf_idf)
  return tfidf

In [ ]:
def tfidf(train, test):
  fitur = fitur_list(train)
  fitur1 = fitur_list(test)
  termfrequency = termfreq(fitur, train)
  termfrequency1 = termfreq(fitur1, test)
  wtf = weightTF(termfrequency)
  wtf1 = weightTF(termfrequency1)
  hasil_idf = idf(wtf)
  for i in range(len(fitur)):
    for j in range(len(wtf1[1])):
      if fitur[i] in fitur1:
        index = fitur1.index(fitur[i])
        wtf[i].append(wtf1[index][j])
      else:
        wtf[i].append(0)
  bobot_docs = tf_idf(hasil_idf, wtf)
  return bobot_docs

In [ ]:
tfidff = tfidf(hasiltrain, hasiltest)

In [ ]:
len(tfidff)

1724

In [ ]:
boosterwords = ['abis', 'agak', 'amat', 'bangat', 'banget', 'beberapa', 'benar-benar', 'betul-betul', 
                'hanya', 'jauh', 'kadang-kadang', 'kali', 'kira-kira', 'kurang', 'lebih', 'mending', 
                'mungkin', 'paling', 'pasti', 'sangat', 'seakan-akan', 'sedikit', 'sekali', 'semakin', 
                'sempurna', 'sungguh', 'tambah', 'telak', 'terkadang', 'terlalu', 'terus-terusan', 'ter'
]

In [ ]:
def booster(text, booster_list):
  f1 = []
  for i in range(len(text)):
    temp = 0
    for n in text[i]:
      if n in booster_list:
        temp +=1
    f1.append(temp)
  return f1

In [ ]:
df3 = pd.read_excel('../../PositiveWords.xlsx')
df3.info()
df4 = pd.read_excel('../../NegativeWords.xlsx')
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1678 entries, 0 to 1677
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1678 non-null   object
dtypes: object(1)
memory usage: 13.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3829 entries, 0 to 3828
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3829 non-null   object
dtypes: object(1)
memory usage: 30.0+ KB


In [ ]:
kamuspos =[]

for row in range(len(df3)):
  kamuspos.append(df3.text[row])

kamusneg =[]

for row in range(len(df4)):
  kamusneg.append(df4.text[row])

In [ ]:
def posneg(text, kamus_positif, kamus_negatif):
  f2 = []
  f3 = []
  for i in range(len(text)):
    temp1 = 0
    temp2 = 0
    for n in text[i]:
      if n in kamus_positif:
        temp1 +=1
      if n in kamus_negatif:
        temp2 +=1
    f2.append(temp1)
    f3.append(temp2)
  return f2,f3


In [ ]:
df5 = pd.read_csv('../../kbbi.csv', encoding = 'unicode_escape', engine ='python')
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28526 entries, 0 to 28525
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   kata    28525 non-null  object
 1   jenis   28526 non-null  object
dtypes: object(2)
memory usage: 445.8+ KB


In [ ]:
def adjposneg(text, kamuspos, kamusneg, kbbi_list):
  f4 = []
  f5 = []
  for i in range(len(text)):
    temp1 = 0
    temp2 = 0
    for n in text[i]:
      if n in kbbi_list:
        if kbbi_list[n] == 'Adjektiva':
          if n in kamuspos:
            temp1 += 1
          if n in kamusneg:
            temp2 += 1
    f4.append(temp1)
    f5.append(temp2)
  return f4,f5


In [ ]:
kbbi = df5.set_index('kata')['jenis'].T.to_dict()

In [ ]:
kbbi['robek']

'Adjektiva'

In [ ]:
def verbposneg(text, kamuspos, kamusneg, kbbi_list):
  f6 = []
  f7 = []
  for i in range(len(text)):
    temp1 = 0
    temp2 = 0
    for n in text[i]:
      if n in kbbi_list:
        if kbbi_list[n] == 'Verba':
          if n in kamuspos:
            temp1 += 1
          if n in kamusneg:
            temp2 += 1
    f6.append(temp1)
    f7.append(temp2)
  return f6,f7

In [ ]:
def advposneg(text, kamuspos, kamusneg, kbbi_list):
  f8 = []
  f9 = []
  for i in range(len(text)):
    temp1 = 0
    temp2 = 0
    for n in text[i]:
      if n in kbbi_list:
        if kbbi_list[n] == 'Adverbia':
          if n in kamuspos:
            temp1 += 1
          if n in kamusneg:
            temp2 += 1
    f8.append(temp1)
    f9.append(temp2)
  return f8,f9

In [ ]:
def adjpercen(text, f2, f3, f4, f5):
  f10 = []
  f11 = []
  for i in range(len(text)):
    temp1 = 0
    temp2 = 0
    if f4[i] > 0:
      temp1 = f4[i]/f2[i]
    if f5[i] > 0:
      temp2 = f5[i]/f3[i]
    f10.append(temp1)
    f11.append(temp2)
  return f10,f11


In [ ]:
def verbpercen(text, f2, f3, f6, f7):
  f12 = []
  f13 = []
  for i in range(len(text)):
    temp1 = 0
    temp2 = 0
    if f6[i] > 0:
      temp1 = f6[i]/f2[i]
    if f7[i] > 0:
      temp2 = f7[i]/f3[i]
    f12.append(temp1)
    f13.append(temp2)
  return f12,f13


In [ ]:
def advpercen(text, f2, f3, f8, f9):
  f14 = []
  f15 = []
  for i in range(len(text)):
    temp1 = 0
    temp2 = 0
    if f8[i] > 0:
      temp1 = f8[i]/f2[i]
    if f9[i] > 0:
      temp2 = f9[i]/f3[i]
    f14.append(temp1)
    f15.append(temp2)
  return f14,f15

In [ ]:
def lexicon(text, boosterword, kamuspos, kamusneg, kbbi):
  fiturlex = []
  f1 = booster(text, boosterword)
  fiturlex.append(f1)
  f2,f3 = posneg(text, kamuspos, kamusneg)
  fiturlex.extend((f2, f3))
  f4,f5 = adjposneg(text, kamuspos, kamusneg,kbbi)
  fiturlex.extend((f4, f5))
  f6,f7 = verbposneg(text, kamuspos, kamusneg, kbbi)
  fiturlex.extend((f6, f7))
  f8,f9 = advposneg(text, kamuspos, kamusneg, kbbi)
  fiturlex.extend((f8, f9))
  f10,f11 = adjpercen(text, f2, f3, f4, f5)
  fiturlex.extend((f10, f11))
  f12,f13 = verbpercen(text, f2, f3, f6, f7)
  fiturlex.extend((f12, f13))
  f14,f15 = advpercen(text, f2, f3, f8, f9)
  fiturlex.extend((f14, f15))
  return fiturlex

In [ ]:
fiturlex = lexicon(doc_tweet, boosterwords, kamuspos, kamusneg, kbbi)

In [ ]:
len(fiturlex[0])

337

In [ ]:
gabungbobot = tfidff + fiturlex

In [ ]:
dff = pd.DataFrame(tfidff)

In [ ]:
tesarray = dff.to_numpy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2134 entries, 0 to 2133
Columns: 312 entries, 0 to 311
dtypes: float64(312)
memory usage: 5.1 MB


In [ ]:
dff.to_excel('bobot.xlsx', index=False, header=False)

In [ ]:
from google.colab import files
files.download('bobot.xlsx') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#SUPPORT VECTOR MACHINE


In [ ]:
kelasdoc =[]

for row in range(len(df)):
  kelasdoc.append(df.Class[row])

In [ ]:
kelastes =[]

for row in range(len(df1)):
  kelastes.append(df1.Class[row])

In [ ]:
dff = pd.read_excel('../../bobot.xlsx', header=None)

In [ ]:
tesarray = dff.to_numpy()

In [ ]:
def kernelLinear(matriks):
  #np.dot (matriks[k],matriks[i])
  kernel = []
  for i in range(len(matriks[0])):
    temp = []
    for j in range(len(matriks[i])):
      jumlah =0
      for k in range(len(matriks)):
        jumlah += matriks[k][j]*matriks[k][i]
      temp.append(jumlah)
    kernel.append(temp)
  return kernel


In [ ]:
liner = kernelLinear(gabungbobot)

In [ ]:
def kernelPolynom(matriks,d):
  #np.dot (matriks[k],matriks[i])
  kernel = []
  for i in range(len(matriks[0])):
    temp = []
    for j in range(len(matriks[i])):
      jumlah =0
      for k in range(len(matriks)):
        jumlah += matriks[k][j]*matriks[k][i]
      kerneli = (jumlah+1)**d
      temp.append(kerneli)
    kernel.append(temp)
  return kernel


In [ ]:
def kernelRBF(matriks, sigma):
  #exp(-gamma * linalg.norm(matriks[j]-matriks[i]) ** 2)
  kernel = []
  for i in range(len(matriks[0])):
    temp = []
    for j in range(len(matriks[i])):
      jumlah =0
      for k in range(len(matriks)):
        jumlah += math.pow(matriks[k][j]-matriks[k][i],2)
      kerneli = math.exp((-1*(jumlah))/(2* (math.pow(sigma,2))))
      temp.append(kerneli)
    kernel.append(temp)
  return kernel


In [ ]:
rbf = kernelRBF(tesarray,5)

In [ ]:
def hessian(kernel, kelas, lambdaval):
  matrikshessian = []
  for i in range(len(kelas)):
    temp = []
    for j in range(len(kelas)):
      hasil = (kelas[i]*kelas[j])*((kernel[i][j])+(math.pow(lambdaval,2)))
      temp.append(hasil)
    matrikshessian.append(temp)
  return matrikshessian


In [ ]:
mh = hessian(liner, kelasdoc, 2)

In [ ]:
def seq_training(hessian, imax, gamma, varC, varE):
  alpha = []
  deltaalpha = []
  E = []
  temp_alpha = []
  for i in range(len(hessian)):
    temp_alpha.append(0)
  alpha.append(temp_alpha)

  for i in range(imax):
    temp_E = []
    temp_delta = []
    temp_alpha = []
    
    for j in range(len(hessian)):
      jml = 0
      for k in range(len(hessian[j])):
        jml = jml + alpha[-1][j] * hessian[j][k]
      temp_E.append(jml)
    E.append(temp_E)

    for l in range(len(hessian)):
      hasil = min(max((gamma*(1-E[-1][l])), (-1*alpha[-1][l])), (varC-alpha[-1][l]))
      temp_delta.append(hasil)
    deltaalpha.append(temp_delta)

    for m in range(len(temp_delta)):
      hasil = alpha[-1][m] + deltaalpha[-1][m]
      temp_alpha.append(hasil)
    alpha.append(temp_alpha)

    if(max(temp_delta)<varE):
      break

  return alpha[-1]

In [ ]:
alpha = seq_training(mh, 5, 0.001, 1, 0.0001)  

In [ ]:
def bias(supportvector, kelas, kernel):
  xPos = 0
  xNeg = 0
  for i in range(len(kelas)):
    if kelas[i] == 1:
      xPos = max(xPos, supportvector[i])
    else:
      xNeg = max(xNeg, supportvector[i])
  alphaPos = supportvector.index(xPos)
  alphaNeg = supportvector.index(xNeg)
  sigmaPos = 0
  sigmaNeg = 0
  kernelPos = []
  kernelNeg = []

  for i in range(len(kelas)):
    temp_kernelPos = []
    temp_kernelNeg =[]

    for j in range(len(kernel[i])):
      pos = kernel[j][alphaPos]
      neg = kernel[j][alphaNeg]
      temp_kernelPos.append(pos)
      temp_kernelNeg.append(neg)
      
    kernelPos = temp_kernelPos
    kernelNeg = temp_kernelNeg
    hasilPos = supportvector[i]*kelas[i]*kernelPos[i]
    hasilNeg = supportvector[i]*kelas[i]*kernelNeg[i]
    sigmaPos= sigmaPos + hasilPos
    sigmaNeg= sigmaNeg + hasilNeg
  bias = -0.5*(sigmaPos+sigmaNeg)
  return bias


In [ ]:
biaz = bias(alpha, kelasdoc, liner)

In [ ]:
def testing(kernel, kelas, alpha, bias):
  kelasuji = []
  sigma = []

  for i in range(len(kelas),len(kernel)):
    ## kelas-kernel
    jml = 0
    for j in range(len(kelas)):
      jml = jml +  (kernel[i][j] * kelas[j] * alpha[j])
    sigma.append(jml)
  
  for i in range(len(sigma)):
    if sigma[i]+bias >=0:
      kelas = 1
    else:
      kelas = -1
    kelasuji.append(kelas)

  return kelasuji

In [ ]:
ujites = testing(liner, kelasdoc, alpha, biaz)

In [ ]:
new_df = pd.DataFrame({'Class': ujites})
df1.update(new_df)
df1

,tweet_text,Class
0,presiden @Jokowi mengatakan dalam keberagaman ...,1
1,Nak SD aja paham ini UU sampah. Pak @jokowi ot...,-1


In [ ]:
def ValConf(predict, actual):
  TP = 0
  FN = 0
  FP = 0
  TN = 0
  for i in range(len(actual)):
    if actual[i] == predict[i]:

      if actual[i] == -1:
        TP +=1
      else:
        TN += 1

    else:

      if actual[i] == -1:
        FN +=1
      else: 
        FP +=1

  return TP, FN, FP, TN

In [ ]:
def eval(TP, FN, FP, TN):
  accuracy = ((TP+TN)/(TP+FN+FP+TN))*100
  if TP+FN >0:
    recall = TP/(TP+FN)
  else:
    recall = 0
  if TP+FP >0:
    precision = TP/(TP+FP)
  else: 
    precision = 0
  if precision+recall>0:
    fmeasure = 2* ((precision*recall)/(precision+recall))
  else:
    fmeasure =0
  return accuracy, precision, recall, fmeasure

In [ ]:
def svm(kernel):
  lambdaval = 0.5
  imax = 50
  gamma = 0.001
  varC = 1
  varE = 0.001
  mh = hessian(kernel, kelasdoc, lambdaval)
  alpha = seq_training(mh, imax, gamma, varC, varE)
  biaz = bias(alpha, kelasdoc, kernel)
  ujites = testing(kernel, kelasdoc, alpha, biaz)
  tp, fn, fp, tn = ValConf(ujites, kelastes)
  acc, preci, rec, fm = eval(tp, fn, fp, tn)
  return acc, preci, rec, fm

In [ ]:
acc, preci, rec, fm = svm(rbf)
acc, preci, rec, fm

(75.49019607843137, 0.9117647058823529, 0.5849056603773585, 0.7126436781609194)

In [ ]:
def main():
  hasiltrain = preprocessing(train)
  hasiltest = preprocessing(test)
  bobottfidf = tfidf(hasiltrain, hasiltest)
  bobotlex = lexicon(doc_tweet, boosterwords, kamuspos, kamusneg, kbbi)
  bobot = bobottfidf + bobotlex
  liner = kernelLinear(bobot)
  acc, preci, rec, fm = svm(liner)
  return acc, preci, rec, fm